In [1]:
# -*- coding: utf-8 -*-
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.10:1.2.0 pyspark-shell'
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")
execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.6 (default, Oct 26 2016 20:30:19)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark import sql
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.feature import Word2Vec
import pandas as pd

    Load the whole datasets

In [3]:
train_path = os.path.join('/labs/lab10data/', 'lab10_train.csv')
item_path = os.path.join('/labs/lab10data/', 'lab10_items.csv')
test_path = os.path.join('/labs/lab10data/', 'lab10_test.csv')
views_programmes = os.path.join('/labs/lab10data', 'lab10_views_programmes.csv')
res_path = os.path.join('/user/artem.moskalets/')

In [4]:
train_fields = [StructField('user_id', LongType(), True),
                StructField('item_id', LongType(), True),
               StructField('purchase', ByteType(), True)]
schema_train = StructType(train_fields)
df_train = spark.read.csv(header = 'true', schema = schema_train, path = train_path, sep=',')

In [5]:
df_train.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- item_id: long (nullable = true)
 |-- purchase: byte (nullable = true)



In [6]:
df_train.show()

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  74107|       0|
|   1654|  89249|       0|
|   1654|  99982|       0|
|   1654|  89901|       0|
|   1654| 100504|       0|
|   1654|  66187|       0|
|   1654|  84350|       0|
|   1654|  92854|       0|
|   1654|  72811|       0|
|   1654|  86876|       0|
|   1654| 102657|       0|
|   1654| 100482|       0|
|   1654|  89677|       0|
|   1654|  99419|       0|
|   1654|  66603|       0|
|   1654|   7363|       0|
|   1654|   1320|       0|
|   1654|  88892|       0|
|   1654|  66671|       0|
|   1654|  75925|       0|
+-------+-------+--------+
only showing top 20 rows



In [7]:
item_fields = [StructField('item_id', LongType(), True),
               StructField('channel_id', DoubleType(), True),
               StructField('datetime_availability_start', TimestampType(), True),
               StructField('datetime_availability_stop', TimestampType(), True),
               StructField('datetime_show_start', TimestampType(), True),
               StructField('datetime_show_stop', TimestampType(), True),
               StructField('content_type', ByteType(), True),
               StructField('title', StringType(), True),
               StructField('year', DoubleType(), True),
               StructField('genres', StringType(), True),
               StructField('region_id', FloatType(), True)]               
schema_item = StructType(item_fields)
df_item = spark.read.csv(header = 'true', schema = schema_item, path = item_path, sep='\t')

In [8]:
names = df_item. select('genres')
names.show()

+--------------------+
|              genres|
+--------------------+
|             Эротика|
|             Эротика|
|             Эротика|
|             Эротика|
|             Эротика|
|             Комедии|
|   Комедии,Мелодрамы|
|Ужасы,Триллеры,Др...|
|Ужасы,Комедии,Фан...|
|Комедии,Мелодрамы...|
|Детективы,Триллер...|
|Фантастика,Боевик...|
|Детективы,Триллер...|
|Детективы,Триллер...|
|Ужасы,Детективы,Т...|
|Комедии,Драмы,Зар...|
|Военные,Боевики,Наши|
|Приключения,Истор...|
|Военные,Драмы,Ист...|
|Комедии,Драмы,Мел...|
+--------------------+
only showing top 20 rows



In [9]:
names = names.withColumn('genres_s',F.split("genres",','))
names.printSchema()

root
 |-- genres: string (nullable = true)
 |-- genres_s: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [10]:
names = names.select(F.explode('genres_s').alias('genres_s')).groupby().agg(F.collect_set('genres_s'))
names_genres = names.rdd.flatMap(lambda x: x).collect()

In [11]:
for i in names_genres:
    for j in sorted(i):
        print j

 сказка
General
Анимация
Аниме
Арт-хаус
Артхаус
Биография
Боевик
Боевики
Вестерн
Видеоигры
Военные
Военный
Детективы
Детские
Детские песни
Для взрослых
Для всей семьи
Для детей
Для самых маленьких
Документальные
Документальный
Драма
Драмы
Западные мультфильмы
Зарубежные
Игры
Исторические
Исторический
Комедии
Комедия
Короткометражки
Короткометражные
Криминал
Кулинария
Мелодрама
Мелодрамы
Мистические
Музыкальные
Музыкальный
Мультсериалы
Мультфильм
Мультфильмы
Мультфильмы в 3D
Мюзиклы
Научная фантастика
Наши
О здоровье
Охота и рыбалка
Передачи
Познавательные
Полнометражные
Приключение
Приключения
Про животных
Прочие
Развивающие
Развлекательные
Реалити-шоу
Романтические
Русские
Русские мультфильмы
Семейные
Семейный
Сериалы
Сказки
Советские
Советское кино
Союзмультфильм
Спорт
Спортивные
Триллер
Триллеры
Ужасы
Фантастика
Фантастические
Фильмы
Фильмы в 3D
Фильмы-спектакли
Фэнтези
Хочу всё знать
Экранизации
Эротика
Юмористические


In [12]:
dictionary = {u'арт-хаус': u'артхаус',             
u'военный': u'военные',\
u'боевики': u'боевик',\
u'вестерн': u'вестерн',\
u'видеоигры': u'видеоигры, игры',\
u'детективы':u'детективы',\
u'документальный': u'документальные',\
u'драмы': u'драма',\
u'исторический': u'исторические',\
u'комедия': u'комедии',\
u'короткометражные':u'короткометражки',
u'мелодрама': u'мелодрамы',\
u'мистические': u'мистические',\
u'музыкальные': u'музыкальный',\
u'мьюзиклы': u'мьюзиклы,музыкальный',\
u'приключение': u'приключения',\
u'романтические': u'мелодрамы',\
u'семейный': u'семейные',\
u'для всей семьи': u'семейные',\
u'советское кино': u'советские, русские',\
u'наши': u'русские',\
u'спортивные': u'спорт',\
u'ужасы': u'триллер, ужасы',\
u'фантастика': u'фантастические',\
u'фильмы в 3d': u'фильмы,3d',\
u'фэнтези': u'фэнтези,фантастические',\
u'научная фантастика': u'научные,фантастические',\
u'хочу всё знать': u'познавательные, развивающие',\
u'познавательные': u'познавательные, развивающие',\
u'детские песни': u'детские,песни',\
u'для детей': u'детские',\
u'для самых маленьких': u'детские',\
u'западные мультфильмы': u'детские, мультфильм,западные',\
u'западные детские' : u'западные, детские',
u'русские детские' : u'русские, детские',             
u'зарубежные': u'зарубежные,западные',
u'мультсериалы': u'детские, мультфильм',\
u'мультфильмы': u'детские,мультфильм',\
u'мультфильм в 3d': u'детские,мультфильм, 3d',\
u'русские мультфильмы': u'детские,мультфильм,русские',\
u'развивающие': u'детские,развивающие',\
u'сказки': u'детские,сказки',\
u'сказка': u'детские,сказки',\
u'триллеры': u'триллер',\
u'союзмультфильм': u'детские,мультфильм',\
u'анимация': u'детские,мультфильм',\
u'эротика': u'эротика, для взрослых'}


In [13]:
test_fields = [StructField('user_id', LongType(), True),
              StructField('item_id', LongType(), True),
              StructField('purchase', ByteType(), True)]
schema_test = StructType(test_fields)
df_test = spark.read.csv(header = 'true', schema = schema_test, path = test_path, sep=',')

In [14]:
df_test.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- item_id: long (nullable = true)
 |-- purchase: byte (nullable = true)



In [15]:
views_programmes_fields = [StructField('user_id', LongType(), True),
                           StructField('item_id', LongType(), True),
                           StructField('ts_start', LongType(), True),
                           StructField('ts_end', LongType(), True),
                           StructField('item_type', StringType(), True)]
schema_views_programmes = StructType(views_programmes_fields)
df_views_programmes = spark.read.csv(header = 'true', schema = schema_views_programmes, path = views_programmes, sep=',')

In [16]:
df_views_programmes.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- item_id: long (nullable = true)
 |-- ts_start: long (nullable = true)
 |-- ts_end: long (nullable = true)
 |-- item_type: string (nullable = true)



    Feature engineering

In [17]:
df_features = df_train.join(df_item, df_train.item_id == df_item.item_id, "inner").drop(df_item.item_id)

In [18]:
df_features = df_features.select('user_id','item_id','purchase','year','genres')

In [19]:
df_features = df_features.select('*', coalesce(df_features["year"], lit(0.0)).alias('year_0'))


In [20]:
df_features = df_features.drop("year").select('*', col('year_0').alias("year")).drop('year_0')

In [21]:
df_features = df_features.fillna('noname')

In [22]:
def gen(films):
    lst =  films.split(',')
    res = ''
    for l in lst:
        l = l.lower().strip()
        for key in dictionary.iterkeys():
            l = l.replace(key, dictionary[key])
        res = res + ','+ l +','
    return res
gen =  udf(gen,StringType())  

In [23]:
df_features = df_features.withColumn('Words', gen(df_features.genres))

In [24]:
df_features.select(df_features.Words).show()

+--------------------+
|               Words|
+--------------------+
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
|,триллер, ужасы,,...|
+--------------------+
only showing top 20 rows



In [25]:
def un(one, two, three, four):
    return str(one)+','+str(two) +','+str(three)+','+(four)
un = udf(un,StringType())
df_features = df_features.withColumn('all_features', un('user_id','item_id','year','words'))

In [26]:
df_features.select('all_features').show(truncate = False)

+------------------------------------------------------------------------+
|all_features                                                            |
+------------------------------------------------------------------------+
|1654,3764,2013.0,,триллер, ужасы,,триллер,,драма,,зарубежные,западные,  |
|510087,3764,2013.0,,триллер, ужасы,,триллер,,драма,,зарубежные,западные,|
|522798,3764,2013.0,,триллер, ужасы,,триллер,,драма,,зарубежные,западные,|
|529632,3764,2013.0,,триллер, ужасы,,триллер,,драма,,зарубежные,западные,|
|566758,3764,2013.0,,триллер, ужасы,,триллер,,драма,,зарубежные,западные,|
|613775,3764,2013.0,,триллер, ужасы,,триллер,,драма,,зарубежные,западные,|
|619378,3764,2013.0,,триллер, ужасы,,триллер,,драма,,зарубежные,западные,|
|625638,3764,2013.0,,триллер, ужасы,,триллер,,драма,,зарубежные,западные,|
|632495,3764,2013.0,,триллер, ужасы,,триллер,,драма,,зарубежные,западные,|
|639612,3764,2013.0,,триллер, ужасы,,триллер,,драма,,зарубежные,западные,|
|651811,3764,2013.0,,трил

In [27]:
df_features.select(df_features.Words).count()

5032624

In [28]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, RegexTokenizer

tokenizer = RegexTokenizer(inputCol="all_features", outputCol="lst", gaps=False, 
                           pattern=ur"[a-zA-Zа-яА-ЯёáéíóúüñÑýÁÉÍÓÚÝ0-9_\-]{2,}")
df_features = tokenizer.transform(df_features)


In [29]:
df_features.select('lst').show(truncate = False)

+--------------------------------------------------------------------------+
|lst                                                                       |
+--------------------------------------------------------------------------+
|[896745, 3764, 2013, триллер, ужасы, триллер, драма, зарубежные, западные]|
|[896756, 3764, 2013, триллер, ужасы, триллер, драма, зарубежные, западные]|
|[896772, 3764, 2013, триллер, ужасы, триллер, драма, зарубежные, западные]|
|[896897, 3764, 2013, триллер, ужасы, триллер, драма, зарубежные, западные]|
|[896954, 3764, 2013, триллер, ужасы, триллер, драма, зарубежные, западные]|
|[896973, 3764, 2013, триллер, ужасы, триллер, драма, зарубежные, западные]|
|[897151, 3764, 2013, триллер, ужасы, триллер, драма, зарубежные, западные]|
|[897162, 3764, 2013, триллер, ужасы, триллер, драма, зарубежные, западные]|
|[897214, 3764, 2013, триллер, ужасы, триллер, драма, зарубежные, западные]|
|[897389, 3764, 2013, триллер, ужасы, триллер, драма, зарубежные, западные]|

In [30]:
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.linalg import Vectors


In [31]:
cv = CountVectorizer(inputCol= "lst", outputCol="features")
model_cv = cv.fit(df_features)

In [32]:
df_features = model_cv.transform(df_features)


In [33]:
df_features.show()

+-------+-------+--------+--------------------+------+--------------------+--------------------+--------------------+--------------------+
|user_id|item_id|purchase|              genres|  year|               Words|        all_features|                 lst|            features|
+-------+-------+--------+--------------------+------+--------------------+--------------------+--------------------+--------------------+
| 899432|   3764|       0|Ужасы,Триллеры,Др...|2013.0|,триллер, ужасы,,...|899432,3764,2013....|[899432, 3764, 20...|(5784,[0,1,4,5,12...|
| 899478|   3764|       0|Ужасы,Триллеры,Др...|2013.0|,триллер, ужасы,,...|899478,3764,2013....|[899478, 3764, 20...|(5784,[0,1,4,5,12...|
| 899541|   3764|       0|Ужасы,Триллеры,Др...|2013.0|,триллер, ужасы,,...|899541,3764,2013....|[899541, 3764, 20...|(5784,[0,1,4,5,12...|
| 899588|   3764|       0|Ужасы,Триллеры,Др...|2013.0|,триллер, ужасы,,...|899588,3764,2013....|[899588, 3764, 20...|(5784,[0,1,4,5,12...|
| 899669|   3764|       0|У

In [34]:
df_features = df_features.drop('genres','Words','user_id','item_id','all_features','genres','year','lst')

In [35]:
df_features.show()

+--------+--------------------+
|purchase|            features|
+--------+--------------------+
|       0|(5784,[0,1,4,5,12...|
|       0|(5784,[0,1,4,5,12...|
|       0|(5784,[0,1,4,5,12...|
|       0|(5784,[0,1,4,5,12...|
|       0|(5784,[0,1,4,5,12...|
|       0|(5784,[0,1,4,5,12...|
|       0|(5784,[0,1,4,5,12...|
|       0|(5784,[0,1,4,5,12...|
|       0|(5784,[0,1,4,5,12...|
|       0|(5784,[0,1,4,5,12...|
|       0|(5784,[0,1,4,5,12...|
|       0|(5784,[0,1,4,5,12...|
|       0|(5784,[0,1,4,5,12...|
|       0|(5784,[0,1,4,5,12...|
|       0|(5784,[0,1,4,5,12...|
|       0|(5784,[0,1,4,5,12...|
|       0|(5784,[0,1,4,5,12...|
|       0|(5784,[0,1,4,5,12...|
|       0|(5784,[0,1,4,5,12...|
|       0|(5784,[0,1,4,5,12...|
+--------+--------------------+
only showing top 20 rows



In [36]:
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel

# Train a RandomForest model.
#rf = RandomForestClassifier(numTrees = 1000, maxDepth = 20, labelCol="purchase", featuresCol="features")

# Chain indexers and forest in a Pipeline
#pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

# Train model.  This also runs the indexers.
#model = rf.fit(df_features)

# Make predictions.

#model.save('RF_lab10_1.sav')
model = RandomForestClassificationModel.load('RF_lab10_1.sav')

In [37]:
df_test.show()

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  94814|    null|
|   1654|  93629|    null|
|   1654|   9980|    null|
|   1654|  95099|    null|
|   1654|  11265|    null|
|   1654|  88896|    null|
|   1654|  67740|    null|
|   1654|  74271|    null|
|   1654|  99871|    null|
|   1654|  78570|    null|
|   1654|  71942|    null|
|   1654|  74367|    null|
|   1654|  98628|    null|
|   1654|  95887|    null|
|   1654|  77795|    null|
|   1654|  75152|    null|
|   1654|  74905|    null|
|   1654|   9068|    null|
|   1654|  72954|    null|
|   1654| 102431|    null|
+-------+-------+--------+
only showing top 20 rows



In [38]:
df_test = spark.read.csv(header = 'true', schema = schema_test, path = test_path, sep=',')
df_test = df_test.join(df_item, df_test.item_id == df_item.item_id, "inner").drop(df_item.item_id)
df_test = df_test.select('user_id','item_id','year','genres')
df_test = df_test.select('*', coalesce(df_test["year"], lit(0.0)).alias('year_0'))
df_test = df_test.drop("year").select('*', col('year_0').alias("year")).drop('year_0')
df_test = df_test.fillna('noname')
df_test = df_test.withColumn('Words', gen(df_test.genres))
df_test = df_test.withColumn('all_features', un('user_id','item_id','year','words'))
df_test = tokenizer.transform(df_test)
df_test = model_cv.transform(df_test)
df_test = df_test.drop('genres','Words','all_features','genres','year','lst','purchase')

In [39]:
df_test.show()

+-------+-------+--------------------+
|user_id|item_id|            features|
+-------+-------+--------------------+
| 900203|   3764|(5784,[0,1,4,5,12...|
| 900223|   3764|(5784,[0,1,4,5,12...|
| 900367|   3764|(5784,[0,1,4,5,12...|
| 900450|   3764|(5784,[0,1,4,5,12...|
| 900597|   3764|(5784,[0,1,4,5,12...|
| 900634|   3764|(5784,[0,1,4,5,12...|
| 900663|   3764|(5784,[0,1,4,5,12...|
| 900697|   3764|(5784,[0,1,4,5,12...|
| 900736|   3764|(5784,[0,1,4,5,12...|
| 900788|   3764|(5784,[0,1,4,5,12...|
| 900816|   3764|(5784,[0,1,4,5,12...|
| 900914|   3764|(5784,[0,1,4,5,12...|
| 900968|   3764|(5784,[0,1,4,5,12...|
| 901589|   3764|(5784,[0,1,4,5,12...|
| 901744|   3764|(5784,[0,1,4,5,12...|
| 901882|   3764|(5784,[0,1,4,5,12...|
| 902209|   3764|(5784,[0,1,4,5,12...|
| 902359|   3764|(5784,[0,1,4,5,12...|
| 902467|   3764|(5784,[0,1,4,5,12...|
| 902480|   3764|(5784,[0,1,4,5,12...|
+-------+-------+--------------------+
only showing top 20 rows



In [40]:
df_test.count()

2156840

In [41]:
2156840

2156840

In [42]:
df_test.show()

+-------+-------+--------------------+
|user_id|item_id|            features|
+-------+-------+--------------------+
| 517612|   3764|(5784,[0,1,4,5,12...|
| 520446|   3764|(5784,[0,1,4,5,12...|
| 523860|   3764|(5784,[0,1,4,5,12...|
| 556825|   3764|(5784,[0,1,4,5,12...|
| 566701|   3764|(5784,[0,1,4,5,12...|
| 575248|   3764|(5784,[0,1,4,5,12...|
| 588378|   3764|(5784,[0,1,4,5,12...|
| 625678|   3764|(5784,[0,1,4,5,12...|
| 627053|   3764|(5784,[0,1,4,5,12...|
| 636572|   3764|(5784,[0,1,4,5,12...|
| 642397|   3764|(5784,[0,1,4,5,12...|
| 668112|   3764|(5784,[0,1,4,5,12...|
| 703514|   3764|(5784,[0,1,4,5,12...|
| 711308|   3764|(5784,[0,1,4,5,12...|
| 736010|   3764|(5784,[0,1,4,5,12...|
| 739215|   3764|(5784,[0,1,4,5,12...|
| 740405|   3764|(5784,[0,1,4,5,12...|
| 742324|   3764|(5784,[0,1,4,5,12...|
| 742986|   3764|(5784,[0,1,4,5,12...|
| 746959|   3764|(5784,[0,1,4,5,12...|
+-------+-------+--------------------+
only showing top 20 rows



In [43]:
predictions = model.transform(df_test)


In [44]:
def proba1(vector):
    from pyspark.ml.linalg import Vectors
   # res = float(vector.dot(Vectors.dense([0.0, 1.0])))
    res = float(vector[1])
    return res
proba1 = udf(proba1, FloatType())

In [45]:
predictions1 = predictions.select('*', proba1('probability').alias('purchase'))

In [46]:
predictions1.show()

+-------+-------+--------------------+--------------------+--------------------+----------+------------+
|user_id|item_id|            features|       rawPrediction|         probability|prediction|    purchase|
+-------+-------+--------------------+--------------------+--------------------+----------+------------+
| 517612|   3764|(5784,[0,1,4,5,12...|[698.553832711902...|[0.99793404673129...|       0.0|0.0020659533|
| 520446|   3764|(5784,[0,1,4,5,12...|[698.553832711902...|[0.99793404673129...|       0.0|0.0020659533|
| 523860|   3764|(5784,[0,1,4,5,12...|[698.553832711902...|[0.99793404673129...|       0.0|0.0020659533|
| 556825|   3764|(5784,[0,1,4,5,12...|[698.525148221201...|[0.99789306888743...|       0.0| 0.002106931|
| 566701|   3764|(5784,[0,1,4,5,12...|[698.538050204086...|[0.99791150029155...|       0.0|0.0020884997|
| 575248|   3764|(5784,[0,1,4,5,12...|[698.553832711902...|[0.99793404673129...|       0.0|0.0020659533|
| 588378|   3764|(5784,[0,1,4,5,12...|[698.553832711902

In [47]:
predictions1.count()

2156840

In [48]:
Res = predictions1.select("user_id","item_id","purchase").\
                    sort("user_id","item_id")


In [49]:
Res.coalesce(1).write.mode("overwrite").option("header", "true").csv("lab10_rfs.csv")
!hadoop fs -getmerge lab10_rfs.csv lab10_4sa.csv
